In [ ]:
import pandas as pd
import numpy as np
import requests

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

import math

# Pacotes
URL = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.1403/dados?formato=json"
resp = requests.get(URL)
df = pd.DataFrame(resp.json())
df['data']  = pd.to_datetime(df['data'], dayfirst=True)
df['valor'] = pd.to_numeric(df['valor'], errors='coerce')
df = df.set_index('data').sort_index().dropna()
series = df['valor'].values.reshape(-1,1)


scaler = MinMaxScaler()
series_scaled = scaler.fit_transform(series)

def create_sequences(data, seq_len):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:i+seq_len])
        y.append(data[i+seq_len])
    return np.array(X), np.array(y)

SEQ_LEN = 12
X_all, y_all = create_sequences(series_scaled, SEQ_LEN)

# CV
tscv = TimeSeriesSplit(n_splits=5)
rmse_list, mae_list, aic_list = [], [], []

for fold, (train_idx, test_idx) in enumerate(tscv.split(X_all), 1):
    X_train, X_test = X_all[train_idx], X_all[test_idx]
    y_train, y_test = y_all[train_idx], y_all[test_idx]
    
    # reshape para [samples, timesteps, features]
    X_train = X_train.reshape(-1, SEQ_LEN, 1)
    X_test  = X_test.reshape(-1, SEQ_LEN, 1)
    
    # Definição do modelo LSTM
    model = Sequential([
        LSTM(50, activation='tanh', input_shape=(SEQ_LEN,1)),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    
    # 6. Treino
    model.fit(X_train, y_train,
              epochs=20,
              batch_size=32,
              verbose=0)
    
    # Previsões 
    y_pred_s = model.predict(X_test).flatten()
    y_true_s = y_test.flatten()
    

    y_pred = scaler.inverse_transform(y_pred_s.reshape(-1,1)).flatten()
    y_true = scaler.inverse_transform(y_true_s.reshape(-1,1)).flatten()
    
    # RMSE e MAE
    rmse = math.sqrt(mean_squared_error(y_true, y_pred))
    mae  = mean_absolute_error(y_true, y_pred)
    
    # AIC
    rss = np.sum((y_true - y_pred)**2)
    n   = len(y_true)
    k   = model.count_params()
    aic = 2*k + n * math.log(rss / n)
    
    rmse_list.append(rmse)
    mae_list.append(mae)
    aic_list.append(aic)
    
    print(f"Fold {fold:>2} → RMSE: {rmse:.4f} | MAE: {mae:.4f} | AIC: {aic:.1f}")

print("\nMédias:")
print(f"  RMSE mean: {np.mean(rmse_list):.4f} ± {np.std(rmse_list):.4f}")
print(f"  MAE  mean: {np.mean(mae_list) :.4f} ± {np.std(mae_list) :.4f}")
print(f"  AIC  mean: {np.mean(aic_list) :.1f} ± {np.std(aic_list) :.1f}")


/home/decio/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Fold  1 → RMSE: 215.6253 | MAE: 166.7289 | AIC: 21869.2


/home/decio/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Fold  2 → RMSE: 671.4999 | MAE: 442.7226 | AIC: 22073.7


/home/decio/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Fold  3 → RMSE: 288.8941 | MAE: 233.3341 | AIC: 21921.9


/home/decio/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Fold  4 → RMSE: 711.4087 | MAE: 581.0525 | AIC: 22084.1


/home/decio/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Fold  5 → RMSE: 893.5946 | MAE: 738.4006 | AIC: 22125.1

Médias:
  RMSE mean: 556.2045 ± 260.2547
  MAE  mean: 432.4478 ± 212.6259
  AIC  mean: 22014.8 ± 100.3


In [ ]:
# pip install tensorflow


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 MB 8.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 37.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 47.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 KB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 32.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 KB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 43.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 45.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 36.7 MB/s